In [ ]:
import sympy as sp
sp.init_printing()
import IPython.display as disp
%matplotlib ipympl
import matplotlib.pyplot as plt
#plt.rcParams['text.usetex'] = True
import numpy as np

In [ ]:
# !!! Indicates a hypothesis (should be studied and considered)

# !!! Mode selection
mode_values = "real" # real or complex values

Wavenumber variables

In [ ]:
# Physical wavenumber
k = sp.Symbol('k',
              #real=True, # !!!
              #positive=True, # !!!
              )

Override inconvenient SymPy behaviour via class surgery

In [ ]:
from sympy.vector import CoordSys3D

from collections.abc import Callable

from sympy.core import S, Dummy, Lambda
from sympy.core.symbol import Str
from sympy.core.symbol import symbols
from sympy.matrices.immutable import ImmutableDenseMatrix as Matrix
from sympy.matrices.matrixbase import MatrixBase
from sympy.vector.scalar import BaseScalar
from sympy.core.containers import Tuple
from sympy.matrices.dense import eye
from sympy.matrices.immutable import ImmutableDenseMatrix
from sympy.vector.coordsysrect import _check_strings
from sympy.vector.vector import BaseVector



# Gives an ambiguous name to coordinates
# x,sys is now x in LaTeX
# Convenient for LaTex printing when working with a unique system
# Otherwise stupid and dangerous
class CoordSys3DUnique(CoordSys3D):

    def __new__(cls, name, transformation=None, parent=None, location=None,
                rotation_matrix=None, vector_names=None, variable_names=None):
        """
        The orientation/location parameters are necessary if this system
        is being defined at a certain orientation or location wrt another.

        Parameters
        ==========

        name : str
            The name of the new CoordSys3D instance.

        transformation : Lambda, Tuple, str
            Transformation defined by transformation equations or chosen
            from predefined ones.

        location : Vector
            The position vector of the new system's origin wrt the parent
            instance.

        rotation_matrix : SymPy ImmutableMatrix
            The rotation matrix of the new coordinate system with respect
            to the parent. In other words, the output of
            new_system.rotation_matrix(parent).

        parent : CoordSys3D
            The coordinate system wrt which the orientation/location
            (or both) is being defined.

        vector_names, variable_names : iterable(optional)
            Iterables of 3 strings each, with custom names for base
            vectors and base scalars of the new system respectively.
            Used for simple str printing.

        """

        name = str(name)
        Vector = sp.vector.Vector
        Point = sp.vector.Point

        if not isinstance(name, str):
            raise TypeError("name should be a string")

        if transformation is not None:
            if (location is not None) or (rotation_matrix is not None):
                raise ValueError("specify either `transformation` or "
                                 "`location`/`rotation_matrix`")
            if isinstance(transformation, (Tuple, tuple, list)):
                if isinstance(transformation[0], MatrixBase):
                    rotation_matrix = transformation[0]
                    location = transformation[1]
                else:
                    transformation = Lambda(transformation[0],
                                            transformation[1])
            elif isinstance(transformation, Callable):
                x1, x2, x3 = symbols('x1 x2 x3', cls=Dummy)
                transformation = Lambda((x1, x2, x3),
                                        transformation(x1, x2, x3))
            elif isinstance(transformation, str):
                transformation = Str(transformation)
            elif isinstance(transformation, (Str, Lambda)):
                pass
            else:
                raise TypeError("transformation: "
                                "wrong type {}".format(type(transformation)))

        # If orientation information has been provided, store
        # the rotation matrix accordingly
        if rotation_matrix is None:
            rotation_matrix = ImmutableDenseMatrix(eye(3))
        else:
            if not isinstance(rotation_matrix, MatrixBase):
                raise TypeError("rotation_matrix should be an Immutable" +
                                "Matrix instance")
            rotation_matrix = rotation_matrix.as_immutable()

        # If location information is not given, adjust the default
        # location as Vector.zero
        if parent is not None:
            if not isinstance(parent, CoordSys3D):
                raise TypeError("parent should be a " +
                                "CoordSys3D/None")
            if location is None:
                location = Vector.zero
            else:
                if not isinstance(location, Vector):
                    raise TypeError("location should be a Vector")
                # Check that location does not contain base
                # scalars
                for x in location.free_symbols:
                    if isinstance(x, BaseScalar):
                        raise ValueError("location should not contain" +
                                         " BaseScalars")
            origin = parent.origin.locate_new(name + '.origin',
                                              location)
        else:
            location = Vector.zero
            origin = Point(name + '.origin')

        if transformation is None:
            transformation = Tuple(rotation_matrix, location)

        if isinstance(transformation, Tuple):
            lambda_transformation = CoordSys3D._compose_rotation_and_translation(
                transformation[0],
                transformation[1],
                parent
            )
            r, l = transformation
            l = l._projections
            lambda_lame = CoordSys3D._get_lame_coeff('cartesian')
            lambda_inverse = lambda x, y, z: r.inv()*Matrix(
                [x-l[0], y-l[1], z-l[2]])
        elif isinstance(transformation, Str):
            trname = transformation.name
            lambda_transformation = CoordSys3D._get_transformation_lambdas(trname)
            if parent is not None:
                if parent.lame_coefficients() != (S.One, S.One, S.One):
                    raise ValueError('Parent for pre-defined coordinate '
                                 'system should be Cartesian.')
            lambda_lame = CoordSys3D._get_lame_coeff(trname)
            lambda_inverse = CoordSys3D._set_inv_trans_equations(trname)
        elif isinstance(transformation, Lambda):
            if not CoordSys3D._check_orthogonality(transformation):
                raise ValueError("The transformation equation does not "
                                 "create orthogonal coordinate system")
            lambda_transformation = transformation
            lambda_lame = CoordSys3D._calculate_lame_coeff(lambda_transformation)
            lambda_inverse = None
        else:
            lambda_transformation = lambda x, y, z: transformation(x, y, z)
            lambda_lame = CoordSys3D._get_lame_coeff(transformation)
            lambda_inverse = None

        if variable_names is None:
            if isinstance(transformation, Lambda):
                variable_names = ["x1", "x2", "x3"]
            elif isinstance(transformation, Str):
                if transformation.name == 'spherical':
                    variable_names = ["r", "theta", "phi"]
                elif transformation.name == 'cylindrical':
                    variable_names = ["r", "theta", "z"]
                else:
                    variable_names = ["x", "y", "z"]
            else:
                variable_names = ["x", "y", "z"]
        if vector_names is None:
            vector_names = ["i", "j", "k"]

        # All systems that are defined as 'roots' are unequal, unless
        # they have the same name.
        # Systems defined at same orientation/position wrt the same
        # 'parent' are equal, irrespective of the name.
        # This is true even if the same orientation is provided via
        # different methods like Axis/Body/Space/Quaternion.
        # However, coincident systems may be seen as unequal if
        # positioned/oriented wrt different parents, even though
        # they may actually be 'coincident' wrt the root system.
        if parent is not None:
            obj = super().__new__(
                cls, Str(name), transformation, parent)
        else:
            obj = super().__new__(
                cls, Str(name), transformation)
        obj._name = name
        # Initialize the base vectors

        _check_strings('vector_names', vector_names)
        vector_names = list(vector_names)
        latex_vects = [(r'\mathbf{\hat{%s}_{%s}}' % (x, name)) for
                           x in vector_names]
        pretty_vects = ['%s_%s' % (x, name) for x in vector_names]

        obj._vector_names = vector_names

        v1 = BaseVector(0, obj, pretty_vects[0], latex_vects[0])
        v2 = BaseVector(1, obj, pretty_vects[1], latex_vects[1])
        v3 = BaseVector(2, obj, pretty_vects[2], latex_vects[2])

        obj._base_vectors = (v1, v2, v3)

        # Initialize the base scalars

        _check_strings('variable_names', vector_names)
        variable_names = list(variable_names)
        # !!! here
        latex_scalars = [(r"\mathbf{{%s}}" % (x)) for
                         x in variable_names]
        pretty_scalars = ['%s_%s' % (x, name) for x in variable_names]

        obj._variable_names = variable_names
        obj._vector_names = vector_names

        x1 = BaseScalar(0, obj, pretty_scalars[0], latex_scalars[0])
        x2 = BaseScalar(1, obj, pretty_scalars[1], latex_scalars[1])
        x3 = BaseScalar(2, obj, pretty_scalars[2], latex_scalars[2])

        obj._base_scalars = (x1, x2, x3)

        obj._transformation = transformation
        obj._transformation_lambda = lambda_transformation
        obj._lame_coefficients = lambda_lame(x1, x2, x3)
        obj._transformation_from_parent_lambda = lambda_inverse

        setattr(obj, variable_names[0], x1)
        setattr(obj, variable_names[1], x2)
        setattr(obj, variable_names[2], x3)

        setattr(obj, vector_names[0], v1)
        setattr(obj, vector_names[1], v2)
        setattr(obj, vector_names[2], v3)

        # Assign params
        obj._parent = parent
        if obj._parent is not None:
            obj._root = obj._parent._root
        else:
            obj._root = obj

        obj._parent_rotation_matrix = rotation_matrix
        obj._origin = origin

        # Return the instance
        return obj

Coordinate variables

In [ ]:
sys = CoordSys3DUnique('sys')
x, y, z = sys.x, sys.y, sys.z



Galerkine variational forms

In [ ]:
from sympy.vector import gradient


def a_G(u, v, xa, xb, ya, yb, za, zb):
    
    # gradients
    grad_u = gradient(u)
    grad_v = gradient(v)
    
    return sp.integrate(grad_u & grad_v, (x, xa, xb), (y, ya, yb), (z, za, zb)) - (k**2 * sp.integrate(u * v, (x, xa, xb), (y, ya, yb), (z, za, zb)))


Grid sizes

In [ ]:
h_ = dict()

for xi in [x, y, z]:
    h_[xi] = dict()
    
    for i in ["-", "+"]:
        h_[xi][i] = sp.Symbol(f'h_{sp.latex(xi)}^{i}')

Grid points centered around origin

In [ ]:
grid_ = dict()

for xi in [x, y, z]:
    grid_[xi] = dict()
    
    grid_[xi]["-1"], grid_[xi]["0"], grid_[xi]["+1"] = -h_[xi]["-"], sp.sympify(0), +h_[xi]["+"]

Point-wise linear shape functions for the three dimensions

In [ ]:
N_1D_ = dict()

for xi in [x, y, z]:
    N_1D_[xi] = dict()
    
    N_1D_[xi]["-1"] = - xi / h_[xi]["-"]
    N_1D_[xi]["0-"] = (xi / h_[xi]["-"]) + 1
    N_1D_[xi]["0+"] = (- xi / h_[xi]["+"]) + 1
    N_1D_[xi]["+1"] = xi / h_[xi]["+"]

Point-wise trilinear shape functions as products

In [ ]:
from itertools import product
from collections import defaultdict


N_3D_ = defaultdict(lambda : defaultdict(dict))

for (i, j, l) in product(["-1", "0-", "0+", "+1"], repeat=3):
    N_3D_[i][j][l] = N_1D_[x][i] * N_1D_[y][j] * N_1D_[z][l]

NumPy cartesian product generator

In [ ]:
def cartesian_product(x, y, *other_arrays):
    if x.ndim < 2:
        x = np.atleast_2d(x).T
    if y.ndim < 2:
        y = np.atleast_2d(y).T

    sx, sy = x.shape, y.shape
    sz = sy[:-1] + sx[:-1] + (sy[-1] + sx[-1],)
    z = np.empty(sz, np.result_type(x, y))

    # Broadcasted assignment
    z[...,:sx[-1]] = x
    z[...,sx[-1]:] = y.reshape(sy[:-1] + (x.ndim-1)*(1,) + (sy[-1],))

    answer = z.reshape((x.shape[0]*y.shape[0], x.shape[1]+y.shape[1]))
    if len(other_arrays) > 0:
        return cartesian_product(answer, *other_arrays)

    return answer

Check that the sum of all functions is $1$ in all of the 8 3D "quadrants"

In [ ]:
# Choose values for simulation
H_ = defaultdict(lambda : dict())
H_[x]['-'], H_[x]['+'], H_[y]['-'], H_[y]['+'], H_[z]['-'], H_[z]['+'] = 0.8, 1.1, 0.7, 1, 0.9, 1.2

# Generate evaluation points
num = 4
XYZ_ = defaultdict(lambda : dict())
for xi in [x, y, z]:
    XYZ_[xi]['-'] = np.linspace(-H_[xi]['-'], 0, num)
    XYZ_[xi]['+'] = np.linspace(0, +H_[xi]['+'], num)

# Evaluate sum function on the quadrant
for (i, j, l) in product(['-', '+'], repeat=3):
    
    print(i,j,l)
    print("-"*10)
    
    # relevant functions in each dimension
    lx = [f'0{i}', f'{i}1']
    ly = [f'0{j}', f'{j}1']
    lz = [f'0{l}', f'{l}1']

    S = sp.sympify(0)
    for (a, b, c) in product(lx, ly, lz):
        print(a, b, c)
        S += N_3D_[a][b][c]
    print("-"*10)
    
    disp.display(S.simplify())
    print("-"*10)
    
    
    S = sp.lambdify(args=[x, y, z, h_[x][i], h_[y][j], h_[z][l]], expr=S)
        
    # evaluation points on this quadrant
    XYZ = cartesian_product(XYZ_[x][i], XYZ_[y][j], XYZ_[z][l])
    
    print(S(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2], H_[x][i], H_[y][j], H_[z][l]))
    print("-"*10)
    
    print("-"*20)

Linear Galerkine stencil equation coefficients

In [ ]:
def nf20(index):
    return '0-' if '-' in index else '0+'

A_G_00_ = defaultdict(lambda : defaultdict(lambda : dict()))

for (i, j, l) in product(["-1", "0", "+1"], repeat=3):
    
    iterr = {
        x: i,
        y: j,
        z: l
    }
    
    A_G_00_[i, j, l] = sp.sympify(0)
    
    # Current node function definitions
    current_node_functions_ = dict()
    for xi in [x, y, z]:
        current_node_functions_[xi] = ["0-", "0+"] if (iterr[xi] == "0") else [iterr[xi]]
    
    for func_x, func_y, func_z in product(*list(current_node_functions_.values())):
        current_node_function = N_3D_[func_x][func_y][func_z]
        relevant_0_function = N_3D_[nf20(func_x)][nf20(func_y)][nf20(func_z)]
        
        # Adequate integration domain
        xa, xb = (grid_[x]['-1'], grid_[x]['0']) if '-' in func_x else (grid_[x]['0'], grid_[x]['+1'])
        ya, yb = (grid_[y]['-1'], grid_[y]['0']) if '-' in func_y else (grid_[y]['0'], grid_[y]['+1'])
        za, zb = (grid_[z]['-1'], grid_[z]['0']) if '-' in func_z else (grid_[z]['0'], grid_[z]['+1'])
        
        A_G_00_[i, j, l] += a_G(current_node_function, relevant_0_function, xa, xb, ya, yb, za, zb)


Check for consistency with the uniform situation

In [ ]:
h_uniform_ = {
    x: sp.symbols('h_x'),
    y: sp.symbols('h_y'),
    z: sp.symbols('h_z')
}

A_G_00_uniform = defaultdict(lambda : defaultdict(lambda : dict()))
for key in A_G_00_:
    A_G_00_uniform[key] = A_G_00_[key].subs([(h_[x]['-'], h_uniform_[x]),
                                             (h_[x]['+'], h_uniform_[x]),
                                             (h_[y]['-'], h_uniform_[y]),
                                             (h_[y]['+'], h_uniform_[y]),
                                             (h_[z]['-'], h_uniform_[z]),
                                             (h_[z]['+'], h_uniform_[z])
                                             ])
    disp.display(key)
    disp.display(A_G_00_uniform[key])
    print('-'*40)